# Laboratorio no calificado: Ingeniería de rasgos manuales
------------------------
 
Bienvenido, durante este laboratorio no graduado vas a realizar ingeniería de características usando TensorFlow y Keras. Teniendo un conocimiento más profundo del problema que estás tratando y proponiendo transformaciones a las características en bruto verás cómo aumenta el poder predictivo de tu modelo. En concreto harás:


1. Definir el modelo usando columnas de características.
2. Utilizar capas lambda para realizar ingeniería de características sobre algunas de ellas.
3. 3. Comparar el historial de entrenamiento y las predicciones del modelo antes y después de la ingeniería de características.

**Nota**: Este laboratorio tiene algunos ajustes en comparación con el código que acaba de ver en las conferencias. El principal es que las variables relacionadas con el tiempo no se utilizan en el modelo de ingeniería de características.

Empecemos.

First, install and import the necessary packages, set up paths to work on and download the dataset.

## Imports

In [ ]:
# Import the packages

# Utilities
import os
import logging

# For visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# For modelling
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras import layers, models

# Set TF logger to only print errors (dismiss warnings)
logging.getLogger("tensorflow").setLevel(logging.ERROR)

## Cargar conjunto de datos taxifare

Para este laboratorio se va a utilizar una versión modificada del conjunto de datos [Taxi Fare dataset](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data), que se ha preprocesado y dividido previamente. 

En primer lugar, cree el directorio en el que se guardarán los datos.



In [ ]:
if not os.path.isdir("/tmp/data"):
    os.makedirs("/tmp/data")

descarga ahora los datos en formato `csv` desde un cubo de almacenamiento en la nube.

In [ ]:
!gsutil cp gs://cloud-training-demos/feat_eng/data/taxi*.csv /tmp/data

Let's check that the files were copied correctly and look like we expect them to.

In [ ]:
!ls -l /tmp/data/*.csv

Everything looks fine. Notice that there are three files, one for each split of `training`, `testing` and `validation`.

## Inspect tha data

Now take a look at the training data.

In [ ]:
pd.read_csv('/tmp/data/taxi-train.csv').head()

Los datos contienen un total de 8 variables.

El `fare_amount` es el objetivo, el valor continuo que vamos a entrenar un modelo para predecir. Esto nos deja con 7 características. 

Sin embargo, este laboratorio se va a centrar en la transformación de los geoespaciales por lo que las características de tiempo `hourofday` y `dayofweek` serán ignorados.

## Crear un pipeline de entrada 

Para cargar los datos para el modelo vas a utilizar una característica experimental de Tensorflow que permite cargar directamente desde un archivo `csv`.

Para ello es necesario definir algunas listas que contengan información relevante del conjunto de datos, como el tipo de las columnas.

In [ ]:
# Especifique qué columna es el objetivo
LABEL_COLUMN = 'fare_amount'

# Especificar columnas numéricas
# Tenga en cuenta que debería crear otra lista con STRING_COLS si 
# tuviera datos de texto pero en este caso todas las características son numéricas
NUMERIC_COLS = ['pickup_longitude', 'pickup_latitude',
                'dropoff_longitude', 'dropoff_latitude',
                'passenger_count', 'hourofday', 'dayofweek']


# Una función para separar características y etiquetas
def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label


# Un método de utilidad para crear un conjunto de datos tf.data a partir de un archivo CSV
def load_dataset(pattern, batch_size=1, mode='eval'):
    dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size)
    
    dataset = dataset.map(features_and_labels)  # features, label
    if mode == 'train':
        # Observe que se utiliza el método de repetición para que este conjunto de datos se repita infinitamente.
        dataset = dataset.shuffle(1000).repeat()
        # aprovechar el multihilo; 1=AUTOTUNE
        dataset = dataset.prefetch(1)
    return dataset

## Crear un Modelo DNN en Keras

Ahora construirás una Red Neuronal simple con las características numéricas como entrada representadas por una capa [`DenseFeatures`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/DenseFeatures) (que produce un Tensor denso basado en las características dadas), dos capas densas con funciones de activación ReLU y una capa de salida con una función de activación lineal (ya que este es un problema de regresión).

Dado que el modelo se define utilizando "columnas de características", la primera capa puede tener un aspecto diferente al habitual. Esto se hace declarando dos diccionarios, uno para las entradas (definidas como capas de entrada) y otro para las características (definidas como columnas de características).

Luego se calcula el tensor `DenseFeatures` pasando las columnas de características al constructor de la capa `DenseFeatures` y pasando las entradas al tensor resultante (esto es más fácil de entender con código):

In [ ]:
def build_dnn_model():
    # input layer
    inputs = {
        colname: layers.Input(name=colname, shape=(), dtype='float32')
        for colname in NUMERIC_COLS
    }

    # feature_columns
    feature_columns = {
        colname: fc.numeric_column(colname)
        for colname in NUMERIC_COLS
    }

    # Constructor para DenseFeatures toma una lista de columnas numéricas
    # y el tensor resultante toma un diccionario de capas Input
    dnn_inputs = layers.DenseFeatures(feature_columns.values())(inputs)

    # dos capas ocultas de 32 y 8 unidades, respectivamente
    h1 = layers.Dense(32, activation='relu', name='h1')(dnn_inputs)
    h2 = layers.Dense(8, activation='relu', name='h2')(h1)

    # la salida final es una activación lineal porque este es un problema de regresión
    output = layers.Dense(1, activation='linear', name='fare')(h2)

    # Create model with inputs and output
    model = models.Model(inputs, output)

    # compile model (Mean Squared Error is suitable for regression)
    model.compile(optimizer='adam', 
                  loss='mse', 
                  metrics=[
                      tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                      'mse'
                  ])

    return model

We'll build our DNN model and inspect the model architecture.

In [ ]:
# Save compiled model into a variable
model = build_dnn_model()

# Plot the layer architecture and relationship between input features
tf.keras.utils.plot_model(model, 'dnn_model.png', show_shapes=False, rankdir='LR')

With the model architecture defined it is time to train it!

## Train the model

You are going to train the model for 20 epochs using a batch size of 32.

In [ ]:
NUM_EPOCHS = 20
TRAIN_BATCH_SIZE = 32 
NUM_TRAIN_EXAMPLES = len(pd.read_csv('/tmp/data/taxi-train.csv'))
NUM_EVAL_EXAMPLES = len(pd.read_csv('/tmp/data/taxi-valid.csv'))

print(f"training split has {NUM_TRAIN_EXAMPLES} examples\n")
print(f"evaluation split has {NUM_EVAL_EXAMPLES} examples\n")

Use the previously defined function to load the datasets from the original csv files.

In [ ]:
# Training dataset
trainds = load_dataset('/tmp/data/taxi-train*', TRAIN_BATCH_SIZE, 'train')

# Evaluation dataset
evalds = load_dataset('/tmp/data/taxi-valid*', 1000, 'eval').take(NUM_EVAL_EXAMPLES//1000)

# Necesita ser especificado ya que el conjunto de datos es infinito 
# Esto ocurre porque se utilizó el método de repetición al crear el conjunto de datos
steps_per_epoch = NUM_TRAIN_EXAMPLES // TRAIN_BATCH_SIZE

# Train the model and save the history
history = model.fit(trainds,
                    validation_data=evalds,
                    epochs=NUM_EPOCHS,
                    steps_per_epoch=steps_per_epoch)

### Visualize training curves

Now lets visualize the training history of the model with the raw features:

In [ ]:
# Function for plotting metrics for a given history
def plot_curves(history, metrics):
    nrows = 1
    ncols = 2
    fig = plt.figure(figsize=(10, 5))

    for idx, key in enumerate(metrics):  
        ax = fig.add_subplot(nrows, ncols, idx+1)
        plt.plot(history.history[key])
        plt.plot(history.history[f'val_{key}'])
        plt.title(f'model {key}')
        plt.ylabel(key)
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')


# Plot history metrics
plot_curves(history, ['loss', 'mse'])

El historial de entrenamiento no parece muy prometedor y muestra un comportamiento errático. Parece que el proceso de entrenamiento tuvo dificultades para atravesar el espacio de alta dimensión que crean las características actuales. 

No obstante, utilicémoslo para la predicción.

Obsérvese que los valores de latitud y longitud deberían girar en torno a (`37`, `45`) y (`-70`, `-78`) respectivamente, ya que son el rango de coordenadas de la ciudad de Nueva York.

In [ ]:
# Define a taxi ride (a data point)
taxi_ride = {
    'pickup_longitude': tf.convert_to_tensor([-73.982683]),
    'pickup_latitude': tf.convert_to_tensor([40.742104]),
    'dropoff_longitude': tf.convert_to_tensor([-73.983766]),
    'dropoff_latitude': tf.convert_to_tensor([40.755174]),
    'passenger_count': tf.convert_to_tensor([3.0]),
    'hourofday': tf.convert_to_tensor([3.0]),
    'dayofweek': tf.convert_to_tensor([3.0]),
}

# Use the model to predict
prediction = model.predict(taxi_ride, steps=1)

# Print prediction
print(f"the model predicted a fare total of {float(prediction):.2f} USD for the ride.")

El modelo predijo que este paseo en particular rondaría los 12 USD. Sin embargo, usted sabe que el rendimiento del modelo no es el mejor, como lo demuestra el historial de entrenamiento. Mejorémoslo utilizando **Ingeniería de características**.

## Mejorar el rendimiento del modelo mediante la ingeniería de rasgos 

En lo sucesivo, sólo se utilizarán características geoespaciales, ya que son las más relevantes a la hora de calcular la tarifa, puesto que este valor depende sobre todo de la distancia recorrida:

In [ ]:
# Drop dayofweek and hourofday features
NUMERIC_COLS = ['pickup_longitude', 'pickup_latitude',
                'dropoff_longitude', 'dropoff_latitude']

Dado que se trata exclusivamente de datos geoespaciales, se crearán algunas transformaciones que tengan en cuenta esta naturaleza geoespacial. Esto ayuda al modelo a hacer una mejor representación del problema en cuestión.

Por ejemplo, el modelo no puede entender mágicamente lo que se supone que representa una coordenada y, dado que los datos proceden únicamente de Nueva York, la latitud y la longitud giran en torno a (`37`, `45`) y (`-70`, `-78`) respectivamente, lo cual es arbitrario para el modelo. Un buen primer paso es escalar estos valores. 

**Nótese que todas las transformaciones se crean definiendo funciones**.

In [ ]:
def scale_longitude(lon_column):
    return (lon_column + 78)/8.

In [ ]:
def scale_latitude(lat_column):
    return (lat_column - 37)/8.

Otro dato importante es que la tarifa de un viaje en taxi es proporcional a la distancia del trayecto. Sin embargo, tal y como están configuradas actualmente las características, el modelo no puede deducir que el par (`latitud_de_recogida`, `longitud_de_recogida`) representa el punto en el que el pasajero inició el trayecto y el par (`latitud_de_salida`, `longitud_de_salida`) representa el punto en el que finalizó el trayecto. Y lo que es más importante, el modelo no es consciente de que la distancia entre estos dos puntos es crucial para predecir la tarifa.

Para solucionar esto, se necesita una nueva característica (que es una transformación de las otras) que proporcione esta información.

In [ ]:
def euclidean(params):
    lon1, lat1, lon2, lat2 = params
    londiff = lon2 - lon1
    latdiff = lat2 - lat1
    return tf.sqrt(londiff*londiff + latdiff*latdiff)

### Aplicar transformaciones

Ahora definiremos la función `transform` que aplicará las funciones de transformación previamente definidas. Para aplicar las transformaciones reales que va a utilizar capas `Lambda` aplicar una función a los valores (en este caso las entradas).


In [ ]:
def transform(inputs, numeric_cols):

    # Make a copy of the inputs to apply the transformations to
    transformed = inputs.copy()

    # Define feature columns
    feature_columns = {
        colname: tf.feature_column.numeric_column(colname)
        for colname in numeric_cols
    }

    # Scaling longitude from range [-70, -78] to [0, 1]
    for lon_col in ['pickup_longitude', 'dropoff_longitude']:
        transformed[lon_col] = layers.Lambda(
            scale_longitude,
            name=f"scale_{lon_col}")(inputs[lon_col])

    # Scaling latitude from range [37, 45] to [0, 1]
    for lat_col in ['pickup_latitude', 'dropoff_latitude']:
        transformed[lat_col] = layers.Lambda(
            scale_latitude,
            name=f'scale_{lat_col}')(inputs[lat_col])

    # add Euclidean distance
    transformed['euclidean'] = layers.Lambda(
        euclidean,
        name='euclidean')([inputs['pickup_longitude'],
                           inputs['pickup_latitude'],
                           inputs['dropoff_longitude'],
                           inputs['dropoff_latitude']])
        
    
    # Add euclidean distance to feature columns
    feature_columns['euclidean'] = fc.numeric_column('euclidean')

    return transformed, feature_columns

## Update the model

Next, you'll create the DNN model now with the engineered (transformed) features.

In [ ]:
def build_dnn_model():
    
    # input layer (notice type of float32 since features are numeric)
    inputs = {
        colname: layers.Input(name=colname, shape=(), dtype='float32')
        for colname in NUMERIC_COLS
    }

    # transformed features
    transformed, feature_columns = transform(inputs, numeric_cols=NUMERIC_COLS)

    # Constructor for DenseFeatures takes a list of numeric columns
    # and the resulting tensor takes a dictionary of Lambda layers
    dnn_inputs = layers.DenseFeatures(feature_columns.values())(transformed)

    # two hidden layers of 32 and 8 units, respectively
    h1 = layers.Dense(32, activation='relu', name='h1')(dnn_inputs)
    h2 = layers.Dense(8, activation='relu', name='h2')(h1)

    # final output is a linear activation because this is a regression problem
    output = layers.Dense(1, activation='linear', name='fare')(h2)

    # Create model with inputs and output
    model = models.Model(inputs, output)

    # Compile model (Mean Squared Error is suitable for regression)
    model.compile(optimizer='adam', 
                  loss='mse', 
                  metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse'), 'mse'])
    
    return model

In [ ]:
# Save compiled model into a variable
model = build_dnn_model()

Let's see how the model architecture has changed.

In [ ]:
# Plot the layer architecture and relationship between input features
tf.keras.utils.plot_model(model, 'dnn_model_engineered.png', show_shapes=False, rankdir='LR')

Este gráfico es muy útil para comprender las relaciones y dependencias entre las características originales y las transformadas.

**Observe que la entrada del modelo consta ahora de 5 características en lugar de las 7 originales, reduciendo así la dimensionalidad del problema.

Entrenemos ahora el modelo que incluye la ingeniería de características.

In [ ]:
history = model.fit(trainds,
                    validation_data=evalds,
                    epochs=NUM_EPOCHS,
                    steps_per_epoch=steps_per_epoch)

Observe que las características `passenger_count`, `hourofday` y `dayofweek` se excluyeron porque se omitieron al definir el canal de entrada.

Ahora vamos a visualizar el historial de entrenamiento del modelo con las características diseñadas. 

In [ ]:
# Plot history metrics
plot_curves(history, ['loss', 'mse'])

Esto parece mucho mejor que el historial de entrenamiento anterior. Ahora las métricas de pérdida y error están disminuyendo con cada epoch y ambas curvas (entrenamiento y validación) están muy cerca la una de la otra. ¡Buen trabajo!

Hagamos una predicción con este nuevo modelo sobre el ejemplo que utilizamos anteriormente. 

In [ ]:
# Use the model to predict
prediction = model.predict(taxi_ride, steps=1)

# Print prediction
print(f"the model predicted a fare total of {float(prediction):.2f} USD for the ride.")

Vaya, ahora el modelo predice una tarifa que es aproximadamente la mitad de lo que predecía el modelo anterior. Parece que el modelo con las características brutas estaba sobreestimando la tarifa por un gran margen.

Observe que aparece una advertencia, ya que el diccionario `taxi_ride` contiene información sobre las características no utilizadas. Puedes suprimirlo redefiniendo `taxi_ride` sin estos valores, pero es útil saber que Keras es lo suficientemente inteligente como para manejarlo por sí mismo.

**Ahora debería tener una comprensión más clara de la importancia y el impacto de realizar ingeniería de características en sus datos. 

Este proceso es muy específico y requiere una gran comprensión de la situación que se está modelando. Por este motivo, se han desarrollado nuevas técnicas que pasan de la ingeniería manual a la automática, y podrás comprobar algunas de ellas en un próximo laboratorio.


**Sigan así.